# Notebook 02: Implementación del Mountain Car Continuous
## Realizado por Víctor Vega Sobral

El objetivo de este notebook es implementar un modelo satisfactorio del problema del coche de la montaña, de la librería de OpenAI gymnasium. 

De esta forma, se podrá tener dos modelos de diferente complejidad en esta práctica y observar diferentes formas de afronar un problema de aprendizaje por refuerzo.



<div style="text-align: center;">
  <img src="images/car.png" alt="Imagen coche " width="350">
</div>


## Importación de bibliotecas necesarias

In [ ]:
import gymnasium as gym
import numpy as np
import random

## Referencias de interés

1. [Documentación oficial de la librería gymnasium](https://gymnasium.farama.org/environments/classic_control/mountain_car_continuous/)

2. [Video de youtube con explicación del problema](https://www.youtube.com/watch?v=_SWnNhM5w-g&t=72s)

3. [Imagen de Research Gate con explicación del problema](https://www.researchgate.net/figure/Continuous-Mountain-Car-environment-a-Original-b-Safety-constrained_fig4_366211448)  

## Explicación del problema

Para una correcta explicación del problema, me remito a traducir la explicación ofrecida por la documentación oficial de OpenAI Gymnasium, añadiendo las modificaciones necesarias para adaptar las explicaciones al formato MarkDown, o añadir aclaraciones:

### Descripción
El Mountain Car MDP  consiste en un automóvil colocado aleatoriamente al fondo de un valle sinusoidal. Las únicas acciones posibles son las aceleraciones que se pueden aplicar al coche en cualquiera de las dos direcciones. El objetivo del MDP es acelerar estratégicamente el coche para alcanzar el estado objetivo en la cima de la colina derecha, haciendo uso de la inercia o momentum que este tenga.

Existen dos versiones de este dominio en Gymnasium: una con acciones discretas y otra con acciones continuas. Esta versión utiliza acciones continuas y será la que se intentará resolver en este notebook.





### Espacio de observaciones 

La observación es un ndarray con forma (2,), donde los elementos corresponden a:

| Num | Observación                    | Mínimo | Máximo | Unidad        |
|-----|--------------------------------|--------|--------|---------------|
| 0   | Posición del coche en el eje x | -Inf   | Inf    | posición (m)  |
| 1   | Velocidad del coche            | -Inf   | Inf    | velocidad (m/s) |


### Espacio de acciones
La acción es un ndarray con forma (1,), que representa la fuerza direccional aplicada al coche. La acción se limita al rango [-1,1] y se multiplica por una potencia de 0.0015.

### Dinámica de transición
Dada una acción, el coche sigue las siguientes dinámicas de transición:


$velocity(t+1) = velocity(t) + force * self.power - 0.0025 * cos(3 * position(t))$

$position(t+1) = position(t) + velocity(t+1)$

Donde:

- **force** es la acción limitada al rango [-1,1].
- **power** es una constante de 0.0015.
- **Las colisiones en cualquiera de los extremos son inelásticas**, con la velocidad establecida en 0 al chocar con la pared. La posición se limita al rango [-1.2, 0.6] y la velocidad al rango [-0.07, 0.07].

### Recompensa
Se recibe una recompensa negativa de $-0.1 * action^2$ en cada paso de tiempo para penalizar acciones de gran magnitud. 

Si el coche alcanza el objetivo, se añade una recompensa positiva de +100 a la recompensa negativa de ese paso.

### Estado inicial
1. La **posición inicial** del coche se asigna aleatoriamente dentro del rango [-0.6, -0.4].
2. La **velocidad inicial** del coche siempre es 0.


### Fin del episodio
El episodio termina si ocurre cualquiera de las siguientes condiciones:

- **Terminación:** La posición del coche es mayor o igual a 0.45 (posición objetivo en la cima de la colina derecha).
- **Truncamiento**: La duración del episodio alcanza los 999 pasos.



## ¿Cómo se va a abordar el problema?

Para intentar solucionar este problema, se va a seguir la siguiente estrategia de resolución:

<div style="text-align: center;">
  <img src="images/explicacion_car.png" alt="Imagen de explicación del coche " width="350">
</div>